In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

from sklearn.model_selection import train_test_split
import xgboost as xgb
import string,re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score

In [55]:
df=pd.read_csv('train.csv')

review=df['Review Text']+df['Review Title']

topic=df['topic']
snl=SentimentIntensityAnalyzer()
df['Star']=review.apply(lambda x:1 if "*" in x else 0)

df['smiley']=review.apply(lambda x:1 if ":)" in x else 0)
df['sad']=review.apply(lambda x:1 if ":(" in x else 0)
df['exp']=review.apply(lambda x:1 if "!" in x else 0)
df['hash']=review.apply(lambda x:1 if "#" in x else 0)
df['comma']=review.apply(lambda x:1 if "," in x else 0)
df['dollar']=review.apply(lambda x:1 if "$" in x else 0)

df['length']=review.apply(lambda x:len(x))
#df['Compound_polarity']=review.apply(lambda x:snl.polarity_scores(x)['compound'])


In [56]:
tfdf=TfidfVectorizer(ngram_range=(1,3),max_df=0.8,min_df=5,max_features=5000)

In [57]:
sw=stopwords.words('english')
ps=PorterStemmer()
def cleatext(text):
    
    text=text.translate(text.maketrans("","",string.punctuation))
    text=[ps.stem(w) for w in text.split(' ') if w not in sw]
    text=' '.join(text)
    return text
review=review.apply(cleatext)

In [58]:
tfdf.fit(review)
X=tfdf.transform(review)

In [59]:
vect=CountVectorizer(ngram_range=(1,3),max_features=10000)
vect.fit(review)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=10000, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [60]:
X_v=vect.transform(review)

In [61]:
y=topic

In [62]:
X.shape

(5959, 5000)

In [63]:
df.drop(['Review Text','Review Title','topic'],axis=1,inplace=True)

In [64]:
df.head()

,Star,smiley,sad,exp,hash,comma,dollar,length
0,0,0,0,0,0,1,0,119
1,0,0,0,0,0,1,0,119
2,0,0,0,1,0,0,0,297
3,0,0,1,0,0,0,0,58
4,0,0,0,0,0,1,0,341


In [65]:
x3=df.values

In [66]:
X=np.hstack([X.toarray(),x3])

In [67]:
X.shape

(5959, 5008)

In [68]:
from sklearn.preprocessing import StandardScaler

In [69]:
sc=StandardScaler()

In [70]:
X=sc.fit_transform(X)

In [71]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [72]:
from sklearn.ensemble import RandomForestClassifier


In [73]:
re =RandomForestClassifier(n_estimators=300)

In [74]:
re.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [75]:
y_pred=re.predict(X_test)



accuracy_score(y_pred,y_test)

0.4203020134228188

In [76]:

xg=xgb.XGBClassifier()
xg.fit(X_train,y_train)





XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)

In [77]:
y_pred=xg.predict(X_test)



accuracy_score(y_pred,y_test)

0.5184563758389261

In [22]:

lg=lgb.LGBMClassifier()

lg.fit(X_train,y_train)

y_pred=lg.predict(X_test)


from sklearn.metrics import accuracy_score

accuracy_score(y_pred,y_test)

0.42197986577181207

In [50]:
X_v.shape

(5959, 185506)

In [51]:
X.shape

(5959, 9102)

In [54]:
X_train,X_test,y_train,y_test=train_test_split(X_v,y,test_size=0.2)

In [55]:
xg=xgb.XGBClassifier()
xg.fit(X_train,y_train)

y_pred=xg.predict(X_test)


from sklearn.metrics import accuracy_score

accuracy_score(y_pred,y_test)

0.5536912751677853

In [41]:
dt=pd.read_csv('test.csv')
dt_review_text=dt['Review Text']
dt_review_title=dt['Review Title']
review_t=dt['Review Text']+dt['Review Title']

In [37]:
review_t=review_t.apply(cleatext)

In [38]:

Xt=tfdf.transform(review_t)

In [61]:
xtt=vect.transform(review_t)

In [63]:
y_pred=xg.predict(xtt)

In [43]:
y_pred1=lg.predict(Xt)

In [42]:

tp=pd.DataFrame({'Review Text':dt_review_text,'Review Title':dt_review_title,'topic':y_pred})
tp.to_csv('Submission_1_XGB.csv',index=False)

In [44]:

tp=pd.DataFrame({'Review Text':dt_review_text,'Review Title':dt_review_title,'topic':y_pred1})
tp.to_csv('Submission_1_lgb.csv',index=False)

In [64]:

tp=pd.DataFrame({'Review Text':dt_review_text,'Review Title':dt_review_title,'topic':y_pred})
tp.to_csv('Submission_2_XGB.csv',index=False)